In [1]:
import numpy as np
import pandas as pd
import time

import requests
import bs4
import json
import re

In [2]:
response = requests.get("https://api.nytimes.com/svc/search/v2/articlesearch.json?q=apple&fq=news_desk:Business&api-key=fO0tDSRQQdU68GkuXbMjt1uA2FYImzVp").json()
docs = response['response']['docs']
url_list = []
for item in docs:
    url_list.append(item['web_url'])
article_list = []
url_list


['https://www.nytimes.com/2020/12/16/technology/facebook-takes-the-gloves-off-in-feud-with-apple.html',
 'https://www.nytimes.com/2020/12/13/business/media/apple-gawker-tim-cook.html',
 'https://www.nytimes.com/2020/12/01/technology/amazon-apple-chips-intel-arm.html',
 'https://www.nytimes.com/2020/12/23/business/dealbook/trump-stimulus-veto.html',
 'https://www.nytimes.com/2020/12/17/business/dealbook/tech-apple-facebook-fight.html',
 'https://www.nytimes.com/2020/12/17/technology/google-antitrust-monopoly.html',
 'https://www.nytimes.com/2020/12/15/technology/big-tech-regulation-europe.html',
 'https://www.nytimes.com/2020/12/14/technology/big-tech-lobbying-europe.html',
 'https://www.nytimes.com/2020/12/09/technology/personaltech/amazon-halo-review.html',
 'https://www.nytimes.com/2020/11/18/technology/apple-app-store-fee.html']

In [3]:
text = []
title = []
abstract = []
for url in url_list:
    time.sleep(0.5)
    headers = {'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36'}
    article = requests.get(url, headers = headers)
    soup = bs4.BeautifulSoup(article.content, 'html.parser')
    article_text_p = soup.find_all('p', attrs={'class': 'css-axufdj evys1bk0'})
    abstract_text_p = soup.find('p', attrs={'class': 'css-w6ymp8 e1wiw3jv0'})
    title_text_h1 = soup.find('h1', attrs={'data-test-id': 'headline'})
    temp = []
    title.append(title_text_h1.text)
    abstract.append(abstract_text_p.text)
    
    for item in article_text_p:
        temp.append(item.text)
    space = ' '
    article_text = space.join(temp)
    text.append(article_text)

In [4]:
len(text)

10

In [5]:
# Word Tokenization to sentences

tokenized_by_sentence = []
for num in range(len(text)):
    text_token = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!|;|”)\s", text[num])
    text_token.insert(0, abstract[num])
    text_token.insert(0, title[num])
    tokenized_by_sentence.append(text_token)

In [6]:
# Goal of this part: Read through all positive / negative tweets, normalize and remove unnecessary words from tweets, then create actual dictionary-like to use for our dataset
import nltk
nltk.download('punkt')
nltk.download('twitter_samples')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yok018\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\yok018\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yok018\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\yok018\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [7]:
from nltk.corpus import twitter_samples

pos_tweet = twitter_samples.tokenized('positive_tweets.json')
neg_tweet = twitter_samples.tokenized('negative_tweets.json')

In [8]:
# Convert all complex part-of-speech to basic words
# List of part-of-speech is in this link: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
# WordNetLemmatizer has a function lemmatize where you can convert complex part of speech words into basic forms
# Things to consider:
#   Remove all unnecessary words from normalized_neg_tweets / normalized_pos_tweets
#   1. Remove mentions(starts with @)
#   2. Remove links (starts with https:// or http:// )
#   3. Remove punctuation (starts with ! or ?)
#   4. Remove Stop-Words (words that do have little to no meaning and does not affect the context of the sentence) to make our dataset more concise
# Note that we are keeping emoji (i.e. :) or :( . That is because these emojis do actually show sentiment of the text context)
# If words are DETERMINERS (DT), COORDINATING CONJUCTIONS (CC), PREPOSITIONS (IN), PERSONAL / POSSESSIVE PRONOUNS (PRP / PRP$), or WH-PRONOUNS (WP) WH-ADVERB(WRB), we remove it (consider as Stop words)
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

normalizer = WordNetLemmatizer()

def determiners(word):
    if word.startswith('@') or word.startswith('!') or word.startswith('?') or word.startswith('https://') or word.startswith('http://'):
        return False
    else:
        return True

normalized_pos_tweets = []
for tweet in pos_tweet:
    sentence = []
    for token, tag in pos_tag(tweet):
        # For Complex Noun words:
        if tag.startswith('NN'):
            new_tag = 'n'
        # For Complex Verb words
        elif tag.startswith('VB'):
            new_tag = 'v'
        # For stop-words
        elif tag.startswith('DT') or tag.startswith('CC') or tag.startswith('IN') or tag.startswith('PRP') or tag.startswith('PRP$') or tag.startswith('WP') or tag.startswith('WRB'):
            continue
        # Every other words, convert them into adjective (pos = 'a')
        else:
            new_tag = 'a'
        sentence.append(normalizer.lemmatize(token, new_tag))
    new_sentence = [ word for word in sentence if determiners(word) ]
    normalized_pos_tweets.append(new_sentence)

# Do Same for Negative Tweets
normalized_neg_tweets = []
for tweet in neg_tweet:
    sentence = []
    for token, tag in pos_tag(tweet):
        # For Complex Noun words:
        if tag.startswith('NN'):
            new_tag = 'n'
        # For Complex Verb words
        elif tag.startswith('VB'):
            new_tag = 'v'
        # For stop-words
        elif tag.startswith('DT') or tag.startswith('CC') or tag.startswith('IN') or tag.startswith('PRP') or tag.startswith('PRP$') or tag.startswith('WP') or tag.startswith('WRB'):
            continue
        # Every other words, convert them into adjective (pos = 'a')
        else:
            new_tag = 'a'
        sentence.append(normalizer.lemmatize(token, new_tag))
    new_sentence = [ word for word in sentence if determiners(word) ]
    normalized_neg_tweets.append(new_sentence)


In [9]:
# Now, store all positive / negative words into dictionary so it can be used as a guide for calculating sentiment for sentences
pos_words_dict = {}
neg_words_dict = {}
